In [14]:
import pandas as pd
import numpy as np

In [15]:
import os
os. chdir("C:/Users/selinederooij/surfdrive/Code/data_processing_library/seizure_data_processing/notebooks")

In [16]:
ann_df = pd.read_csv("../data/annotations_TLE_patients.csv")
ann_df.head()

,index,Orig_Index,File_No,Patient,Filename,Seizure_Start,Seizure_Stop,Seizure_Type,train/dev,seizure_id,seizure_duration,file_id,T1_T2
0,52,53,28,705,/train/02_tcp_le/007/00000705/s001_2003_11_26/...,NaN,NaN,NaN,train,NaN,NaN,28,True
1,53,54,29,705,/train/02_tcp_le/007/00000705/s001_2003_11_26/...,NaN,NaN,NaN,train,NaN,NaN,29,True
2,116,117,73,77,/train/02_tcp_le/000/00000077/s001_2003_03_18/...,NaN,NaN,NaN,train,NaN,NaN,73,True
3,138,139,85,1349,/train/02_tcp_le/013/00001349/s002_2004_09_21/...,NaN,NaN,NaN,train,NaN,NaN,85,True
4,139,140,86,1317,/train/02_tcp_le/013/00001317/s002_2004_05_25/...,NaN,NaN,NaN,train,NaN,NaN,86,True


In [17]:
from src.config import FEATURES_DIR
feat_df = pd.read_parquet(FEATURES_DIR + "features.parquet")
feat_df.head()

,epoch,annotation,start_time,stop_time,min|T1,min|T2,max|T1,max|T2,nzc|T1,nzc|T2,...,mean_power_beta|T2,mean_power_HF|T1,mean_power_HF|T2,spectral_entropy|T1,spectral_entropy|T2,line_length|T1,line_length|T2,sample_entropy|T1,sample_entropy|T2,filename
0,0,-1,0.0,2.0,57.0,58.0,57.0,58.0,1,1,...,0.323706,0.029357,0.029357,0.352865,0.352595,104.044428,103.794001,0.007909,0.008063,/train/02_tcp_le/015/00001543/s001_2004_07_14/...
1,1,-1,2.0,4.0,93.0,85.0,93.0,85.0,1,1,...,0.323714,0.029357,0.029357,0.341046,0.339222,105.873275,106.500597,0.002287,0.003280,/train/02_tcp_le/015/00001543/s001_2004_07_14/...
2,2,-1,4.0,6.0,95.0,92.0,94.0,91.0,2,2,...,0.323704,0.029357,0.029357,0.347019,0.343520,189.639396,189.770344,0.006545,0.007596,/train/02_tcp_le/015/00001543/s001_2004_07_14/...
3,3,-1,6.0,8.0,89.0,94.0,90.0,94.0,1,1,...,0.323712,0.029357,0.029357,0.346084,0.345651,101.675915,101.242494,0.004542,0.004535,/train/02_tcp_le/015/00001543/s001_2004_07_14/...
4,4,-1,8.0,10.0,74.0,80.0,75.0,80.0,3,4,...,1.783831,0.041464,0.152353,0.329116,0.313819,436.393297,518.228620,0.061199,0.041820,/train/02_tcp_le/015/00001543/s001_2004_07_14/...


In [18]:
seiz_df = ann_df[~ann_df["Seizure_Type"].isna()].copy()
seiz_df.head()

,index,Orig_Index,File_No,Patient,Filename,Seizure_Start,Seizure_Stop,Seizure_Type,train/dev,seizure_id,seizure_duration,file_id,T1_T2
5,167,168,98,1795,/train/02_tcp_le/017/00001795/s001_2004_08_05/...,87.6337,114.068,ABSZ,train,98.0,26.4343,98,True
6,168,169,98,1795,/train/02_tcp_le/017/00001795/s001_2004_08_05/...,786.5480,806.372,ABSZ,train,99.0,19.8240,98,True
14,350,351,232,2806,/train/02_tcp_le/028/00002806/s002_2004_12_23/...,30.8880,102.888,FNSZ,train,175.0,72.0000,232,True
15,351,352,232,2806,/train/02_tcp_le/028/00002806/s002_2004_12_23/...,140.1240,175.124,FNSZ,train,176.0,35.0000,232,True
16,352,353,232,2806,/train/02_tcp_le/028/00002806/s002_2004_12_23/...,232.2680,254.668,FNSZ,train,177.0,22.4000,232,True


In [19]:
set_df = pd.DataFrame(columns=["filename", "seizure_id", "annotation"])
set_df["filename"] = feat_df["filename"].copy()
set_df["annotation"] = feat_df["annotation"].copy()
set_df["seizure_id"] = None

In [20]:
for file in seiz_df["Filename"].unique():
    temp = feat_df[feat_df["filename"] == file]
    temp = temp[temp["annotation"] == 1]
    file_seiz = seiz_df[seiz_df["Filename"] == file]
    for i, row in temp.iterrows():
        start_time = row["start_time"]
        stop_time = row["stop_time"]
        seiz_id = file_seiz.loc[
            (
                (file_seiz["Seizure_Start"].between(start_time, stop_time))
                & (file_seiz["Seizure_Stop"] > stop_time)
            )
            | (
                (file_seiz["Seizure_Stop"] > stop_time)
                & (file_seiz["Seizure_Start"] < start_time)
            )
            | (
                (file_seiz["Seizure_Start"] < start_time)
                & (file_seiz["Seizure_Stop"].between(start_time, stop_time))
            ),
            "seizure_id",
        ].values[0]
        set_df.loc[i, "seizure_id"] = int(seiz_id)

In [ ]:
ann_df = ann_df.loc[:, ["Filename", "Patient"]].drop_duplicates()
set_df = pd.merge(set_df, ann_df, left_on="filename", right_on="Filename", how="left")

In [ ]:
# remove 'Filename' column
set_df.drop(columns=["Filename"], inplace=True)

In [ ]:
set_df["index"] = set_df.index

In [ ]:
set_df["group_id"] = set_df["seizure_id"]
bckg_df = set_df[set_df["seizure_id"].isna()].copy()
for patient in bckg_df["Patient"].unique():
    temp = bckg_df[bckg_df["Patient"] == patient]
    seiz_ids = set_df.loc[set_df["Patient"] == patient, "seizure_id"].dropna().unique()
    num_seiz = len(seiz_ids)
    indices = temp["index"].values
    # divide indices into groups by seiz_id randomly
    indices = np.random.permutation(indices)
    groups = np.array_split(indices, num_seiz)
    for i, seiz in enumerate(seiz_ids):
        set_df.loc[set_df["index"].isin(groups[i]), "group_id"] = seiz

In [ ]:
set_df.to_parquet(FEATURES_DIR + "val_groups.parquet")